In [1]:
from tkinter import *
import cv2 
from PIL import Image, ImageTk, ImageDraw
import numpy as np
import time
import math
import sys
from pyfirmata import Arduino

In [2]:
# Seting up pyfirmata params
# board = Arduino('COM12')

# # Left motor pins
# enableLeftPin = board.get_pin('d:11:p')
# input1LeftPin = board.get_pin('d:13:o')
# input2LeftPin = board.get_pin('d:12:o')

# # Right motor pins
# enableRightPin = board.get_pin('d:10:p')
# input3RightPin = board.get_pin('d:9:o')
# input4RightPin = board.get_pin('d:8:o')


In [3]:
# Setting Aruco params

# Pruned aruco type dict. See PoseEstimation.ipynb for full list
ARUCO_DICT = {"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL}

arucoDict = cv2.aruco.getPredefinedDictionary(ARUCO_DICT["DICT_ARUCO_ORIGINAL"])

arucoParams = cv2.aruco.DetectorParameters()


intrinsic_camera = np.array(((933.15867, 0, 657.59),(0,933.1586, 400.36993),(0,0,1)))
distortion = np.array((-0.43948,0.18514,0,0))

In [4]:
# Functions for Positioning



# This function will take an image and overlay evenly spaced circles within a specified box area.
# Also it appends 
#Note: 
    # origin(0,0) of picture is at top-left corner
    # x-axis increases to the right
    # y-axis increases downward
def overlay_circles_on_image(image, circle_radius, circle_color, points):
    #points is a list of tuples

    
    # Convert numpy array to PIL Image
    image_pil = Image.fromarray(image)
    draw = ImageDraw.Draw(image_pil)
    str1 = "Points type: " + str(type(points)) + " "
    # print(str1)
    # Draw circles at each point
    for point in points:
        left_up_point = (point[0] - circle_radius, point[1] - circle_radius)
        right_down_point = (point[0] + circle_radius, point[1] + circle_radius)
        draw.ellipse([left_up_point, right_down_point], fill=circle_color)
    
    # Convert the modified PIL Image back to a numpy array
    modified_image = np.array(image_pil)
    # Return the modified image
    return modified_image

# calculates the relative rotation angles between two markers represented by their rotation 
# vectors rvec1 and rvec2. ( vec1 and rvec2 come from pose_estimation function.

def angle_between_markers(rvec1, rvec2):
    # Calculate relative rotation vectors
    relative_rvec1 = rvec1 - np.array([0, 0, 0])  # Assume origin is at (0, 0, 0)
    relative_rvec2 = rvec2 - np.array([0, 0, 0])  # Assume origin is at (0, 0, 0)

    # Convert relative rotation vectors to rotation matrices
    rotation_matrix1, _ = cv2.Rodrigues(relative_rvec1)
    rotation_matrix2, _ = cv2.Rodrigues(relative_rvec2)

    # Compute the rotation difference between the two markers
    relative_rotation_matrix = np.dot(rotation_matrix2, np.linalg.inv(rotation_matrix1))
        
    # Convert rotation matrix to Euler angles
    relative_euler_angles, _ = cv2.Rodrigues(relative_rotation_matrix)
     
    # Convert to Angles in x, y, and z
    theta_x = np.arctan2(relative_rotation_matrix[2, 1], relative_rotation_matrix[2, 2])
    theta_y = np.arctan2(-relative_rotation_matrix[2, 0], np.sqrt(relative_rotation_matrix[2, 1]**2 + relative_rotation_matrix[2, 2]**2))
    theta_z = np.arctan2(relative_rotation_matrix[1, 0], relative_rotation_matrix[0, 0])
    
    # Convert to degrees
    theta_x_deg = np.degrees(theta_x)
    theta_y_deg = np.degrees(theta_y)
    theta_z_deg = np.degrees(theta_z)
    
    # Make it a single matrix
    Theta = np.array([theta_x_deg, theta_y_deg, theta_z_deg])
        
    return Theta

# Finds the absolute distance between two aruco markers
def distance_between_markers(tvec1, tvec2):
    distance = np.linalg.norm(tvec1 - tvec2)
    return distance

def draw_axis(img, rvec, tvec, camera_matrix, dist_coeffs, length):
    axis_points = np.float32([[0,0,0], [length,0,0], [0,length,0], [0,0,length]]).reshape(-1,3)

    # Project axis points to the image plane
    img_points, _ = cv2.projectPoints(axis_points, rvec, tvec, camera_matrix, dist_coeffs)

    # Convert image points to integers
    img_points = np.round(img_points).astype(int)

    # Draw lines
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[1].ravel()), (0,0,255), 2)  # x-axis (red)
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[2].ravel()), (0,255,0), 2)  # y-axis (green)
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[3].ravel()), (255,0,0), 2)  # z-axis (blue)

    return img

def aruco_display(corners, ids, rejected, image):  
	if len(corners) > 0:
		
		ids = ids.flatten()
		
		for (markerCorner, markerID) in zip(corners, ids):
			
			corners = markerCorner.reshape((4, 2))
			(topLeft, topRight, bottomRight, bottomLeft) = corners
			
			topRight = (int(topRight[0]), int(topRight[1]))
			bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
			bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
			topLeft = (int(topLeft[0]), int(topLeft[1]))

			cv2.line(image, topLeft, topRight, (0, 255, 0), 2)
			cv2.line(image, topRight, bottomRight, (0, 255, 0), 2)
			cv2.line(image, bottomRight, bottomLeft, (0, 255, 0), 2)
			cv2.line(image, bottomLeft, topLeft, (0, 255, 0), 2)
			
			cX = int((topLeft[0] + bottomRight[0]) / 2.0)
			cY = int((topLeft[1] + bottomRight[1]) / 2.0)
			cv2.circle(image, (cX, cY), 4, (0, 0, 255), -1)
			
			cv2.putText(image, str(markerID),(topLeft[0], topLeft[1] - 10), cv2.FONT_HERSHEY_SIMPLEX,
				0.5, (0, 255, 0), 2)
			
	return image

# Newest Pose Estimation that outputs Aruco Coordinates
def pose_estimation(frame, aruco_dict_type, matrix_coefficients, distortion_coefficients):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_dict_type)  # Define aruco_dict here
    parameters = cv2.aruco.DetectorParameters()

    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    # Call aruco_display to draw markers and their IDs
    frame = aruco_display(corners, ids, rejected_img_points, frame)
    distance_vector = None;
    x_rel = None
    y_rel = None
    rot_aboutZ= None;
    center_x = None  # Default value
    center_y = None  # Default value
    
    if ids is not None and 150 in ids:
        marker_index = np.where(ids == 150)[0][0]
        marker_corners = corners[marker_index][0]
        center_x = int(np.mean(marker_corners[:, 0]))
        center_y = int(np.mean(marker_corners[:, 1]))
        #print("Pixel Coordinates of Marker 0 (Center):",(center_x, center_y))

    if len(corners) > 1:
       # print("There are two or more markers")
        # Estimate pose for the first marker
        rvec1, tvec1, _ = cv2.aruco.estimatePoseSingleMarkers(corners[0], 0.175, matrix_coefficients, distortion_coefficients)

        # Estimate pose for the second marker
        rvec2, tvec2, _ = cv2.aruco.estimatePoseSingleMarkers(corners[1], 0.175, matrix_coefficients, distortion_coefficients)
        
        # Calculating Relative Position Vectors
        distance_vector_nested_list = -(tvec1-tvec2);
        array = np.array(distance_vector_nested_list)
        flattened_array = array.flatten()
        x_rel = flattened_array[0]
        y_rel = flattened_array[1]
        #distance = distance_between_markers(tvec1.squeeze(), tvec2.squeeze())
        #print("Absolute Distance between markers:", distance)
        #print("Distance vector between markers:", distance_vector)
        
        # Calculating Relative Rotations
        Theta = angle_between_markers(rvec1.squeeze(), rvec2.squeeze()) 
        rot_aboutZ= -Theta[2]
        #print(f"Relative Rotation Angle around X-axis (degrees): {Theta[0]}")
        #print(f"Relative Rotation Angle around Y-axis (degrees): {Theta[1]}")
        #print(f"Relative Rotation Angle around Z-axis (degrees): {Theta[2]}")
        
        cv2.aruco.drawDetectedMarkers(frame, corners)
        draw_axis(frame, rvec1, tvec1, matrix_coefficients, distortion_coefficients, 0.1)
        draw_axis(frame, rvec2, tvec2, matrix_coefficients, distortion_coefficients, 0.1)

    return frame, (x_rel,y_rel), rot_aboutZ, (center_x, center_y)



In [5]:
# defining  functions
def setMotorSpeeds(leftSpeed, rightSpeed):
    str1 = str(leftSpeed) + " / " + str(rightSpeed)
    # print(str1)
        # # Set left motor direction
        # if leftSpeed >= 0:
        #     input1LeftPin.write(1)
        #     input2LeftPin.write(0)
        # else:
        #     input1LeftPin.write(0)
        #     input2LeftPin.write(1)
        #     leftSpeed = -leftSpeed
      
        # # Set right motor deirection
        # if rightSpeed >= 0:
        #     input3RightPin.write(1)
        #     input4RightPin.write(0)
        # else:
        #     input3RightPin.write(0)
        #     input4RightPin.write(1)
        #     rightSpeed = -rightSpeed
    
        # enableLeftPin.write(leftSpeed/255)
        # enableRightPin.write(rightSpeed/255)

def calcMotorSpeed_PID_TupleIn(currentPoseTuple, desiredPositionTuple, CCW_ANGLE_DIRECTION = -1, Y_UP_SENSE = -1, integralError = 0, derivativeError = 0, verbose = False, howVerbose = 0):
    '''Parameters
    ----------
    robotPoseTuple : Tuple of floats
        (x, y, phi) Tuple describing the robot's current position and heading: .
    desiredPointTuple : Tuple of floats
        (x_desired, y_desired) Position of robot's current point it's tracking towards.
    CCW_ANGLE_DIRECTION: 1 or -1, default is 1
        Input -1 if rotating CCW decreases angle reading
    integralError : float, optional
        Integral Error, whatever way you like to calculate. The default is 0.
    derivativeError : float, optional
        Derivative Error, whatever way you like to calculate. The default is 0.

    Returns
    -------
    motorSpeedTuple : Tuple of Floats
        (leftSpeed, rightSpeed, integralError) speeds as float vary from -255 to 255.
    '''
    # Process Input
    x = currentPoseTuple[0];
    y = currentPoseTuple[1];
    phi = currentPoseTuple[2];
    x_desired = desiredPositionTuple[0];
    y_desired = desiredPositionTuple[1];
    # Constant Parameters: 
    kp = 1.5 #proportional constant
    ki = 0.0 #integralError constant
    kd = 0.0 #derivativeError constant
    ANGLE_OF_GOING_RIGHT = 90.0; #angle (deg) the robot detects when moving right. Set to -90 if up is 0, CCW is positive.
    DEFAULT_SPEED = 255 #the regular speed the robot runs at
    MAX_SPEED = 255.0 #exactly 255
    MIN_SPEED = -255.0;
    ######

    # Same code as before
    dx = x_desired - x;
    dy = y_desired - y;
    if (Y_UP_SENSE == -1):
        dy = -dy;
    
    leftSpeed = DEFAULT_SPEED
    rightSpeed = DEFAULT_SPEED
    angleToDesired = math.degrees(math.atan2(dy, dx)); #
    currentAngle = phi - ANGLE_OF_GOING_RIGHT
    
    #Positive error means need to move CCW
    error = angleToDesired - currentAngle;
    error = CCW_ANGLE_DIRECTION * error;
    integralError += error;
    
    u = kp * error + ki * integralError + kd * derivativeError

    leftSpeedRAW = leftSpeed - u
    rightSpeedRAW = rightSpeed + u    
    leftSpeed = max(MIN_SPEED, min(MAX_SPEED, leftSpeed - u)) 
    rightSpeed = max(MIN_SPEED, min(MAX_SPEED, rightSpeed + u))
    motorSpeedTuple = (leftSpeed, rightSpeed, integralError)
    
    if (verbose and howVerbose > 0):
        str1 = "Motor PID Verbose Output:"
        if (howVerbose >= 1):
            str1 += "\n\tRaw Current Pose \t(x,y,phi) = " + str(currentPoseTuple)
            str1 += "\n\tRaw Desired Pos\t(x_des,y_des) = " + str(desiredPositionTuple)
            str1 += "\n\tCalculated current angle = \t" + str(round(currentAngle,3)) + " degrees"
        if (howVerbose >= 2):
            # Define a tuple
            str1 += "\n\tError \t\t= " + str(round(error, 3)) + " degrees"
            str1 += "\n\tCCW direction sense = " + str(CCW_ANGLE_DIRECTION)
            str1 += "\n\tControl u \t = " + str(round(u, 4))
            str1 += "\n\tRaw Speed \t = "+ str(round(leftSpeedRAW, 3)) + "\t/ " + str(round(rightSpeedRAW, 3))
        str1 += "\n\tOutput Speed = " + str(round(leftSpeed, 2)) + "\t/ " + str(round(rightSpeed, 2))
        print(str1)
    return motorSpeedTuple #return a tuple, of the new motor speeds


def generate_lawnmower_path(xTopLeft, yTopLeft, xBottomRight, yBottomRight, L, ds=2):
    '''
    Parameters
    ----------
    xTopLeft : float
        X coordinate of Top Left (or bottom left) point of bounding box.
    yTopLeft : float
        Y coordinate of Top Left (or bottom left) point of bounding box.
    xBottomRight : float
        X coordinate of Bottom Right (or top right) point of bounding box.
    yBottomRight : float
        X coordinate of Bottom Right (or top right) point of bounding box.
    L : float
        Distance between zig zags. Input the width of the robot or something.
    ds : float, optional
        The distance between points. The default is 2 units.

    Returns
    -------
    points : List of tuples, where each row/tuple is (x,y) of that point along path
        DESCRIPTION.

    '''
    points = []
    x_current = xTopLeft
    y_current = yTopLeft

    width = xBottomRight - xTopLeft
    height = yBottomRight - yTopLeft

    # Ensure that height is positive, assuming the direction is downward
    if height < 0:
        height = -height
        y_direction = -1
    else:
        y_direction = 1

    direction = 1  # 1 for right, -1 for left

    while (y_direction == 1 and y_current < yBottomRight) or (y_direction == -1 and y_current > yBottomRight):
        # Horizontal movement
        if direction == 1:
            while x_current < xBottomRight:
                points.append((x_current, y_current))
                x_current += ds
                if x_current >= xBottomRight:
                    x_current = xBottomRight
                    points.append((x_current, y_current))
                    break
        else:
            while x_current > xTopLeft:
                points.append((x_current, y_current))
                x_current -= ds
                if x_current <= xTopLeft:
                    x_current = xTopLeft
                    points.append((x_current, y_current))
                    break

        # Vertical movement
        y_next = y_current + y_direction * L
        if y_direction == 1 and y_next >= yBottomRight:
            y_next = yBottomRight
        elif y_direction == -1 and y_next <= yBottomRight:
            y_next = yBottomRight
        while (y_direction == 1 and y_current < y_next) or (y_direction == -1 and y_current > y_next):
            y_current += y_direction * ds
            if (y_direction == 1 and y_current > y_next) or (y_direction == -1 and y_current < y_next):
                y_current = y_next
            points.append((x_current, y_current))

        direction *= -1

    return points
def plot_things(path, origin, robotPosition, robotDirection, rWidth):
    """
    Plot several things using Matplotlib:
    1. List of tuples (path) as individual points.
    2. Tuple (origin) as a red 'x'.
    3. Arrow with tail at robotPosition and length rWidth in the direction of robotDirection.

    Parameters:
        path (list of tuples or numpy arrays): List of tuples (x, y) or numpy arrays to be plotted as individual points.
        origin (tuple or numpy array): Tuple (x, y) or numpy array to be plotted as a red 'x'.
        robotPosition (tuple or numpy array): Tuple (x, y) or numpy array specifying the position of the robot.
        robotDirection (float): Direction of the robot in degrees.
        rWidth (float): Length of the arrow representing the robot's direction.
    """
    # Convert input tuples to numpy arrays
    if isinstance(path, list):
        path = np.array(path)
    if isinstance(origin, tuple):
        origin = np.array(origin)
    if isinstance(robotPosition, tuple):
        robotPosition = np.array(robotPosition)
    
    # Extract x and y coordinates from path
    x_path, y_path = path[:, 0], path[:, 1]
    
    # Plot the path as individual points
    plt.plot(x_path, y_path, 'bo', label='Path')
    
    # Plot the origin as a red 'x'
    plt.plot(origin[0], origin[1], 'rx', label='Origin')
    
    # Calculate the end point of the arrow
    x_end = robotPosition[0] + rWidth * np.cos(np.radians(robotDirection))
    y_end = robotPosition[1] + rWidth * np.sin(np.radians(robotDirection))
    
    # Plot the arrow with smaller head
    plt.arrow(robotPosition[0], robotPosition[1], x_end - robotPosition[0], y_end - robotPosition[1],
              head_width=2, head_length=3, fc='g', ec='g', label='Robot')
    plt.plot(robotPosition[0], robotPosition[1], 'go')

    # Set plot limits
    plt.xlim(min(x_path.min(), origin[0], robotPosition[0]), max(x_path.max(), origin[0], robotPosition[0]))
    plt.ylim(max(y_path.max(), origin[1], robotPosition[1]), min(y_path.min(), origin[1], robotPosition[1]))  # Inverted y-axis
    
    # Set plot title and labels
    plt.title('Plot of Path, Origin, and Robot')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    # Add legend
    plt.legend()
    
    # Invert y-axis
    plt.gca().invert_yaxis()
    
    # Show the plot
    plt.grid(True)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()

In [6]:
def plotPath(xTopLeft, yTopLeft, xBottomRight, yBottomRight, L, ds):
    # xTopLeft = 0
    # yTopLeft = 0
    # xBottomRight = 100
    # yBottomRight = 200
    # L = 20
    # ds = 2
    
    lawnmower_points = generate_lawnmower_path(xTopLeft, yTopLeft, xBottomRight, yBottomRight, L, ds, 0)
    
    for i in range(1,10):
        thisPoint = lawnmower_points[i]
        # print(thisPoint)
    
    
    # Plotting the points with matplotlib
    plt.figure(figsize=(8, 8))
    for idx, (x, y) in enumerate(lawnmower_points):
        plt.plot(x, y, 'bo')  # Plot the point
        plt.text(x, y, str(idx), fontsize=9, ha='right')  # Label the point with its index
    
    plt.plot(xTopLeft, yTopLeft, 'rx')
    plt.plot(xBottomRight, yBottomRight, 'gx')
    
    # Set plot limits
    plt.xlim(xTopLeft - 10, xBottomRight + 10)
    plt.ylim(min(yTopLeft, yBottomRight) - 10, max(yTopLeft, yBottomRight) + 10)
    
    # Set labels and title
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Discretized Lawnmower Path with Point Indices')
    plt.grid(True)
    plt.show()


In [7]:
def checkTupleAndSetDefault(thisTuple, defaultTuple, verbose = False):
    if thisTuple[0] is None:
        return defaultTuple
    
    
    
    if (thisTuple is None or not isinstance(thisTuple, tuple)):
        if verbose:
            print("it's a none")
        return defaultTuple
    return thisTuple
def current_milli_time():
    return round(time.time()*100)

In [ ]:
# Function for capturing video frame and displaying it.
# Schedules itself to run again in 10ms
def open_camera():
    global initiate_cleaning, cleaning, pts, i, x1, y1, x2, y2, xhome, yhome, lastI, integralError, lastPhi, lastPos, lastOrigin
    # Capture the video frame by frame 
    _, frame = vid.read()

    output, Dist_Vector_Rel, Rot_about_Z, origin_coords = pose_estimation(frame, ARUCO_DICT["DICT_ARUCO_ORIGINAL"], intrinsic_camera, distortion)
    if Dist_Vector_Rel[0] is None or origin_coords[0] is None: 
        Dist_Vector_Rel = lastPos
        Rot_about_Z = lastPhi
        origin_coords = lastOrigin
    else:
        lastPos = Dist_Vector_Rel
        lastPhi = Rot_about_Z
        lastOrigin = origin_coords 
  
    try:
        frame = output
    except:
        print("whoops, these aren't compatible. frame and output are not")

    # Initialize variables with default values
    x1_int = x1
    y1_int = y1
    x2_int = x2
    y2_int = y2
    xhome_int = xhome
    yhome_int = yhome

    # Draw the bounding box
    try:
        x1_int = int(x1_var.get())
        y1_int = int(y1_var.get())
        x2_int = int(x2_var.get())
        y2_int = int(y2_var.get())
    except ValueError:
        pass  # Use default values if the inputs are not valid integers

    cv2.rectangle(frame, (x1_int, y1_int), (x2_int, y2_int), (255, 0, 0), 2)
    
    # Draw the home position
    try:
        xhome_int = int(xhome_var.get())
        yhome_int = int(yhome_var.get())
        cv2.circle(frame, (xhome_int, yhome_int), 5, (0, 255, 0), -1)
    except ValueError:
        pass  # Handle the case where the inputs are not valid integers

    if len(pts) > 0:
        pts_temp = pts
        # for i, pt in enumerate(pts):
        #     pts_temp[i] = tuple(x + y for x, y in zip(pts[i], lastOrigin))
        # print(pts_temp)
    
        frame = overlay_circles_on_image(frame, 3, (255,0,0,255), pts_temp[i:i])
        frame = overlay_circles_on_image(frame, 2, (0,0,0,255), pts_temp[i+1:])
    # cv2.putText(frame, "awesome", (30, 50), font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=1, thickness=2)

    # Convert image from one color space to other 
    opencv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
    
    # Capture the latest frame and transform to image 
    captured_image = Image.fromarray(opencv_image)

    # Convert captured image to photoimage 
    photo_image = ImageTk.PhotoImage(image=captured_image)

    # Displaying photoimage in the label 
    label_widget.photo_image = photo_image

    # Configure image in the label 
    label_widget.configure(image=photo_image)

    robotWidth = 20
    pts_temp = pts
    
    # Run the following if the "Clean" button has been clicked
    if initiate_cleaning and not cleaning:
        pts = generate_lawnmower_path(x1_int, y1_int, x2_int, y2_int, 0.75 * robotWidth, ds=robotWidth / 10.0) + [(xhome_int, yhome_int)]
        # print("Original")
        # print(pts)
        # for i, pt in enumerate(pts):
        #     pts_temp[i] = tuple(a - b for a, b in zip(pt, lastOrigin))

        # pts = pts_temp
        # print("modified")
        # print(pts)
        i = 0
        integralError = 0
        lastI = i;
        initiate_cleaning = False
        cleaning = True

    if cleaning:
        TRACKING_THRESHOLD = int(0.1*robotWidth)
        #thisPointToTrack  = tuple(a + b for a, b in zip(pts[i], lastOrigin))
        thisPointToTrack  = pts[i]
        
        try:
            currentPoseTuple = Dist_Vector_Rel + (Rot_about_Z,)
        except:
            print("couldn't update pose tuple")
            pass
        lastPoseTuple = lastPos + (lastPhi,)
        
        # print(thisPointToTrack)
        # print(currentPoseTuple)
        currentPoseTuple = lastPoseTuple
        #checkTupleAndSetDefault(currentPoseTuple, lastPoseTuple, verbose = True)
        dx = thisPointToTrack[0] - currentPoseTuple[0]
        # print(thisPointToTrack)
        # print(currentPoseTuple)
        dy = thisPointToTrack[1] - currentPoseTuple[1]
        distToPoint = math.sqrt( dx**2 + dy**2)

        
        # # Define the text you want to display
        # thisString = "Hello, OpenCV!"
        
        # # Define the position (top-left corner)
        # position = (10, 30)  # x, y coordinates
        
        # # Define the font, scale, color, and thickness
        # font = cv2.FONT_HERSHEY_SIMPLEX
        # font_scale = 1
        # font_color = (255, 255, 255)  # white color in BGR
        # thickness = 2
        
        # # Put the text on the image
        # cv2.putText(frame, thisString, position, font, font_scale, font_color, thickness)
        
        # # Display the image
        # cv2.imshow("Image with Text", frame)
        
        # # Wait for a key press and close the image window
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
                
        # if dist to pts(i) < threshold
        # if i == length(pts): cleaning = False, motor speeds = 0
        # else: i++, reset error vals
        if (distToPoint < TRACKING_THRESHOLD): #robot is close enough to point
            if (i == len(pts)):
                cleaning = False
                print("Stop Cleaning")
                print("Done cleaning, returned home")
            else:
                i += 1
                integralError = 0
                #print("resetting integral error")
        if(i == len(pts) - 1):
            print("returning home")
        if (lastI != i):
            print("Changing to new point")
            print(currentPoseTuple)
            lastI = i

        # run PID
        # (leftSpeed, rightSpeed) = calcMotorSpeed_PID(robotPos, pt, 0, 0)
        if (current_milli_time() % 10 < 1):
            verb = True
            print("Printing")
            strT = "t = " + str(current_milli_time())
            print(strT)
        else:
            verb = False
        # (leftSpeed, rightSpeed, integralError) = calcMotorSpeed_PID_TupleIn(currentPoseTuple, thisPointToTrack, integralError, verbose = verb, howVerbose = 2)
        (leftSpeed, rightSpeed, integralError) = calcMotorSpeed_PID_TupleIn(currentPoseTuple, thisPointToTrack, CCW_ANGLE_DIRECTION = -1, Y_UP_SENSE = -1, integralError = 0, derivativeError = 0, verbose = verb, howVerbose = 2)
        
        
        #integralError is modified by calcMotorSpeed_PID
        # print(leftSpeed)
        # print(rightSpeed)
        setMotorSpeeds(leftSpeed, rightSpeed)

    # Repeat the same process after every 10 milliseconds 
    label_widget.after(10, open_camera)

# Function to handle mouse click events
def on_mouse_click(event):
    global x1, y1, x2, y2, xhome, yhome, bounding_box_selected, home_selected, click_count

    if bounding_box_selected:
        if click_count == 0:
            x1, y1 = event.x, event.y
            x1_var.set(x1)
            y1_var.set(y1)
            click_count += 1
        elif click_count == 1:
            x2, y2 = event.x, event.y
            x2_var.set(x2)
            y2_var.set(y2)
            click_count = 0
            bounding_box_selected = False
    elif home_selected:
        xhome, yhome = event.x, event.y
        xhome_var.set(xhome)
        yhome_var.set(yhome)
        click_count = 0
        home_selected = False

# Function to start bounding box selection
def choose_bounding_box():
    global bounding_box_selected, home_selected, click_count
    bounding_box_selected = True
    home_selected = False
    click_count = 0

# Function to start home selection
def choose_home():
    global home_selected, bounding_box_selected
    home_selected = True
    bounding_box_selected = False
    click_count = 0

# Function to set cleaning initiation
def start_cleaning():
    global initiate_cleaning
    initiate_cleaning = True

# Function to stop the robot
def stop_robot():
    global cleaning
    setMotorSpeeds(0, 0)
    cleaning = False

# Define a video capture object 
vid = cv2.VideoCapture(0) 

# Declare the width and height in variables 
width, height = 854, 480

# Set the width and height 
# adjust to match phone resolution
vid.set(cv2.CAP_PROP_FRAME_WIDTH, width) 
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, height) 

# Create a GUI app 
app = Tk() 
app.state('zoomed')

# Bind the app with Escape keyboard to 
# quit app whenever pressed 
# app.bind('<Escape>', lambda e: app.quit())

# Function to release the camera when the window is closed
def on_closing():
    vid.release()
    app.destroy()
    # board.sp.close()

app.protocol("WM_DELETE_WINDOW", on_closing)

# Create a label and display it on app 
label_widget = Label(app) 
label_widget.pack() 
label_widget.bind('<Button-1>', on_mouse_click)

# Create a frame to hold all controls
control_frame = Frame(app)
control_frame.pack()

# Open Camera button on the left
open_camera_button = Button(control_frame, text="Open Camera", command=open_camera)
open_camera_button.grid(row=0, column=0, padx=10, pady=10)

# Create input fields and labels for user-defined cleaning boundaries
x1_var = StringVar()
y1_var = StringVar()
x2_var = StringVar()
y2_var = StringVar()
xhome_var = StringVar()
yhome_var = StringVar()

# Home coordinates and button
home_frame = Frame(control_frame)
home_frame.grid(row=0, column=1, padx=10, pady=10)
home_button = Button(home_frame, text="Choose Home", command=choose_home)
home_button.grid(row=0, column=0, rowspan=2, padx=10)

label_xhome = Label(home_frame, text="x home:")
label_xhome.grid(row=0, column=1)
entry_xhome = Entry(home_frame, textvariable=xhome_var)
entry_xhome.grid(row=0, column=2)

label_yhome = Label(home_frame, text="y home:")
label_yhome.grid(row=1, column=1)
entry_yhome = Entry(home_frame, textvariable=yhome_var)
entry_yhome.grid(row=1, column=2)

# Bounding box coordinates and button
bbox_frame = Frame(control_frame)
bbox_frame.grid(row=0, column=2, padx=10, pady=10)
bounding_box_button = Button(bbox_frame, text="Choose Bounding Box", command=choose_bounding_box)
bounding_box_button.grid(row=0, column=0, rowspan=2, padx=10)

label_x1 = Label(bbox_frame, text="x1:")
label_x1.grid(row=0, column=1)
entry_x1 = Entry(bbox_frame, textvariable=x1_var)
entry_x1.grid(row=0, column=2)

label_y1 = Label(bbox_frame, text="y1:")
label_y1.grid(row=1, column=1)
entry_y1 = Entry(bbox_frame, textvariable=y1_var)
entry_y1.grid(row=1, column=2)

label_x2 = Label(bbox_frame, text="x2:")
label_x2.grid(row=0, column=3)
entry_x2 = Entry(bbox_frame, textvariable=x2_var)
entry_x2.grid(row=0, column=4)

label_y2 = Label(bbox_frame, text="y2:")
label_y2.grid(row=1, column=3)
entry_y2 = Entry(bbox_frame, textvariable=y2_var)
entry_y2.grid(row=1, column=4)

# Start Cleaning button on the right
start_button = Button(control_frame, text="Start Cleaning", command=start_cleaning)
start_button.grid(row=0, column=3, padx=10, pady=10)

# Stop Robot button on the right
stop_button = Button(control_frame, text="Stop Robot", command=stop_robot)
stop_button.grid(row=0, column=4, padx=10, pady=10)

# Initialize state as not cleaning
cleaning = False  # Usually False
initiate_cleaning = False
returning_home = False

# Initialize array of desired pts and all control variables
pts = []
i = 0
lastPos = (0, 0)
lastPhi = 0
lastOrigin = (0, 0)

# Variables for bounding box and home selection
bounding_box_selected = False
home_selected = False
click_count = 0
x1, y1, x2, y2 = 0, 0, 0, 0
xhome, yhome = 0, 0

# Create an infinite loop for displaying app on screen
app.mainloop()

Printing
t = 171698607990
Motor PID Verbose Output:
	Raw Current Pose 	(x,y,phi) = (0, 0, 0)
	Raw Desired Pos	(x_des,y_des) = (161, 160)
	Calculated current angle = 	-90.0 degrees
	Error 		= -45.178 degrees
	CCW direction sense = -1
	Control u 	 = -67.7677
	Raw Speed 	 = 322.768	/ 187.232
	Output Speed = 255.0	/ 187.23
Printing
t = 171698608090
Motor PID Verbose Output:
	Raw Current Pose 	(x,y,phi) = (0, 0, 0)
	Raw Desired Pos	(x_des,y_des) = (161, 160)
	Calculated current angle = 	-90.0 degrees
	Error 		= -45.178 degrees
	CCW direction sense = -1
	Control u 	 = -67.7677
	Raw Speed 	 = 322.768	/ 187.232
	Output Speed = 255.0	/ 187.23
Printing
t = 171698608170
Motor PID Verbose Output:
	Raw Current Pose 	(x,y,phi) = (0, 0, 0)
	Raw Desired Pos	(x_des,y_des) = (161, 160)
	Calculated current angle = 	-90.0 degrees
	Error 		= -45.178 degrees
	CCW direction sense = -1
	Control u 	 = -67.7677
	Raw Speed 	 = 322.768	/ 187.232
	Output Speed = 255.0	/ 187.23
Printing
t = 171698608180
Motor PID V